# 자연어 기반 기후기술분류(DACON)  

재미로 해보는 데이콘 경진대회 데이터를 이용한 분류 작업..공부한다고 생각하며 진행하자

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install konlpy

In [29]:
cd /content/drive/MyDrive/dataset/NLP/dacon

/content/drive/MyDrive/dataset/NLP/dacon


In [30]:
# 일단 생각나는 패키지 전부 했음. 추후 필요한 것 추가 혹은 필요 없는 것 삭제 할 것
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import re
import random

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Conv1D, Dropout, concatenate, LSTM, GRU, GlobalMaxPool1D, Input, Embedding
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import konlpy
# 한국어에 대한 색인어 추출을 목적으로 하는 Okt 를 사용하기로 일단 결정...!
from konlpy.tag import Okt, Komoran, Kkma

## Train / Test Dataset 정리  
각 데이터를 확인 한 후, 필요한 column을 따라 훈련을 위한 데이터 셋을 새로 생성한다.

In [31]:
train_file = pd.read_csv('./train.csv')
test_file = pd.read_csv('./test.csv')
label_file = pd.read_csv('./labels_mapping.csv')

In [ ]:
label_file['소분류'].to_list()

In [32]:
train_file = train_file.drop(['index','계속과제여부','제출년도','내역사업명','사업_부처명','사업명'],axis = 1)

In [48]:
train_file.head()

,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0


In [33]:
train_file = train_file.dropna()

In [47]:
test_file.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드
0,174304,2016,경제협력권산업육성,산업통상자원부,신규,자동차융합부품,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 차체 점용접부의 품질 검사를 위한 64채널 무선 기반 C-Scan 탐촉자 개발\...,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...,○ 기술적 파급효과\n\n - 본 연구에서 개발된 R-FSSW 접합 기술은 기존 ...,"마찰교반점용접, 비파괴 검사, 초음파 탐상, 씨 스캔, 용접 품질 평가","Friction Stir Spot Welding, Non-destructive ev..."
1,174305,2018,개인기초연구(과기정통부)(R&D),과학기술정보통신부,계속,신진연구(총연구비5천이상~1.5억이하),다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...,본 연구는 물리학에서 중요한 대상인 다입자계를 묘사하는 모델방정식의 정당성을 보장하...,"다체계 방정식,동역학의 안정성,양자역학,고전역학,평균장 극한,고전극한,비상대론적 극한","many particle system,stability of dynamics,qua..."
2,174306,2017,개인기초연구(미래부),과학기술정보통신부,계속,자유공모,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,"저출생체중아는 정상 체중 출생아에 비하여 운동, 정신, 행동 및 정서발달장애가 빈번...",본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,본 연구는 저출생체중아 가족 대상의 지역사회기반의 추후관리 사업에 대한 구체적인 모...,"저출생체중아,추후관리,조기개입,자조모임,가정방문","lowbirthweightinfant,followup,earlyinterventio..."
3,174307,2018,산학연협력기술개발(R&D),중소벤처기업부,신규,전략 협력(지역유망),폴리싱용 5축 CNC 정밀 마이크로 시스템 개발,최근 초정밀 공작기계 시장의 트렌드를 살펴보면 다음과 같다.\n\n\n(1) 자동차...,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...,(1) 가공 공정 및 소요시간 단축\n\n\n 초정밀금형 가공시 정삭 후 추가적...,"초정밀가공,폴리싱 가공,마이크로,경면가공,난삭재가공","superprecision,hiprecision,micro,mirror like f..."
4,174308,2019,이공학학술연구기반구축(R&D),교육부,계속,지역대학우수과학자지원사업(1년~5년),다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발,본 연구의 최종목표는 다결정 금속재료의 미세조직구조를 고려한 확률론적 응력부식균열(...,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차...",본 연구의 중요성 및 활용가능성은 다음과 같다.\n\n ∙ 기존의 재료전체 ave...,"결정구조,응력부식균열,확률론적모델","Crystalline structure,Stress corrosion crackin..."


In [49]:
train_ds = train_file[['과제명','label']]
test_ds = test_file[['과제명']]

In [37]:
train_ds.shape

(171138, 2)

# 데이터 전처리 과정  
내부의 특수기호, 공백 등을 삭제하고 훈련을 위한 단어만을 추출하는 과정

In [ ]:
okt = Okt()
komoran = Komoran()
kkma = Kkma()

In [ ]:
# 전처리 과정
def preprocessing(text, tool):
  text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text) # 한글 및 공백을 제외한 문자 제거(영문자, 특수문자)
  nouns = tool.nouns(text)
  return nouns

In [ ]:
# 일단 과제명으로 테스트 한 후 진행...
print(train_file['과제명'][0])

유전정보를 활용한 새로운 해충 분류군 동정기술 개발


In [ ]:
nouns = komoran.nouns(train_file['과제명'][0])
print(komoran.morphs(train_file['과제명'][0]))
print(komoran.pos(train_file['과제명'][0]))
print(nouns)

['유전', '정보', '를', '활용', '하', 'ㄴ', '새롭', 'ㄴ', '해충', '분류군', '동정', '기술', '개발']
[('유전', 'NNP'), ('정보', 'NNG'), ('를', 'JKO'), ('활용', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETM'), ('새롭', 'VA'), ('ㄴ', 'ETM'), ('해충', 'NNP'), ('분류군', 'NNP'), ('동정', 'NNP'), ('기술', 'NNP'), ('개발', 'NNG')]
['유전', '정보', '활용', '해충', '분류군', '동정', '기술', '개발']


In [ ]:
nouns = kkma.nouns(train_file['과제명'][0])
print(kkma.pos(train_file['과제명'][0]))
print(nouns)

[('유전', 'NNG'), ('정보', 'NNG'), ('를', 'JKO'), ('활용', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('해충', 'NNG'), ('분류', 'NNG'), ('군', 'NNG'), ('동정', 'NNG'), ('기술', 'NNG'), ('개발', 'NNG')]
['유전', '유전정보', '정보', '활용', '새', '해충', '분류', '분류군', '군', '동정', '동정기술', '기술', '개발']


In [ ]:
stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']
komoran = Komoran()
clean_train_text=[]
clean_test_text=[]

for text in tqdm.tqdm(train_ds['과제명']):
    try:
        clean_train_text.append(preprocessing(text, komoran))
    except:
        clean_train_text.append([])

for text in tqdm.tqdm(test_ds['과제명']):
    if type(text) == str:
        clean_test_text.append(preprocessing(text, komoran))
    else:
        clean_test_text.append([])

100%|██████████| 43576/43576 [00:25<00:00, 1690.11it/s]


In [ ]:
def max_len(text):
  max = 0
  for i in text:
    if len(i) > max :
      max = len(i)

  return max

maxlen = max_len(clean_train_text)

In [ ]:
clean_train_text[0]

['유전', '정보', '활용', '해충', '분류군', '동정', '기술', '개발']

In [ ]:
#텐서플로의 전처리 모듈을 활용해 토크나이징 객체를 만든 후 인덱스 벡터로 전환
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_train_text)

train_sequences=tokenizer.texts_to_sequences(clean_train_text)
test_sequences=tokenizer.texts_to_sequences(clean_test_text)
word_vocab=tokenizer.word_index

#패딩 처리
train_inputs=pad_sequences(train_sequences, maxlen=maxlen, padding='post')
test_inputs=pad_sequences(test_sequences, maxlen=maxlen, padding='post')

In [ ]:
# label
labels = np.array(train_ds['label'])

# 학습
기본적으로 CNN을 사용해 주제 분류를 하고자 한다.

In [ ]:
# Modeling
Total_size = len(word_vocab) + 1
embedding_size = 128
dropout = 0.5
epoch = 300

input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(Total_size, embedding_size, input_length=maxlen)(input_layer)
dropout_emb = Dropout(rate = dropout)(embedding_layer)

conv1 = Conv1D(filters = 128, kernel_size = 3, padding = 'valid',activation = tf.nn.gelu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters = 128, kernel_size = 4, padding = 'valid', activation = tf.nn.gelu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(filters = 128, kernel_size = 5, padding = 'valid', activation = tf.nn.gelu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1,pool2,pool3])
hidden = Dense(128, activation=tf.nn.gelu)(concat)
dropout_hidden = Dropout(rate=dropout)(hidden)
logits = Dense(46, name = 'logits')(dropout_hidden)
predictions = Dense(46,activation=tf.nn.softmax)(logits)

In [ ]:
model = Model(inputs = input_layer, outputs = predictions)
model.compile(optimizer = tf.keras.optimizers.Nadam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
Stopper = EarlyStopping(monitor='accuracy', mode = 'max', verbose = 1, patience = 10)

In [ ]:
model.fit(train_inputs, labels, epochs = epoch, verbose = 2, validation_split= 0.2, callbacks = Stopper)

# 제출 파일

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
pred = model.predict(test_inputs)
pred=tf.argmax(pred, axis=1)
sample_submission['label']=pred
sample_submission.to_csv('./result_0725_1.csv', index=False)